In [ ]:
# Standard library imports
import glob
import os
from pathlib import Path
import shutil
import urllib.request
import zipfile

# Third-party library imports
import gcsfs
import gdown
import matplotlib.pyplot as plt
import numpy as np
from numba import njit
import pandas as pd
import requests
import seaborn as sns
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import tqdm
from typing import List, Dict, Tuple


In [ ]:
def download_and_unzip_google_drive_folder(folder_url: str, output_folder: str, zip_file_name: str):
    """
    Download a folder from Google Drive and unzip its contents.

    :param folder_url: The URL of the Google Drive folder to download.
    :param output_folder: The local directory to save the downloaded folder.
    :param zip_file_name: The name of the zip file expected to be in the downloaded folder.
    """
    # Ensure the output directory exists
    Path(output_folder).mkdir(parents=True, exist_ok=True)

    gdown.download_folder(folder_url, output=output_folder, quiet=False)

    # Define the path to the expected zip file
    zip_path = Path(output_folder) / zip_file_name
    # Unzip the file
    if zip_path.exists():
        print(f"Unzipping {zip_file_name}...")
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(output_folder)
        print("Unzip complete.")

        # Delete the zip file after unzipping
        zip_path.unlink()
        print(f"Deleted {zip_file_name}.")

        dir2_folder = os.path.join(output_folder, "TEST_CAMEL_DATASET")

        for item in os.listdir(dir2_folder):
            source_path = os.path.join(dir2_folder, item)
            destination_path = os.path.join(output_folder, item)

            # Check if the destination path already exists
            if not os.path.exists(destination_path):
                shutil.move(source_path, output_folder)
        # Now, delete dir2 completely
        shutil.rmtree(dir2_folder)


        # Check for and delete the _MACOSX folder if it exists
        macosx_path = Path(output_folder) / '__MACOSX'
        if macosx_path.exists():
            shutil.rmtree(macosx_path)
            print("Deleted _MACOSX folder.")

    else:
        print(f"{zip_file_name} not found in {output_folder}.")

# Example usage
folder_url = "https://drive.google.com/drive/folders/17mBun1X9abd4n4J2T7l8EuffykDoeqvS?usp=drive_link"
output_folder = "/content/CAMEL_DATASET"
zip_file_name = "TEST_CAMEL_DATASET.zip"
download_and_unzip_google_drive_folder(folder_url, output_folder, zip_file_name)

In [ ]:
def summarize_dir_structure(directory, depth=0, max_depth=3):
    """
    Summarize the directory structure up to a specified depth.

    Parameters:
    - directory: Path object representing the directory to summarize.
    - depth: Current depth in the directory structure.
    - max_depth: Maximum depth to traverse to in the directory structure.
    """
    # Base case: if the current depth exceeds the max depth, stop recursion.
    if depth > max_depth:
        return

    indent = " " * 4 * depth  # Indentation based on current depth
    if directory.is_dir():
        if depth == 0:  # Print root directory without indent
            print(f"{directory.name}")
        else:  # Print subdirectories with indent
            print(f"{indent}└── {directory.name}")

        # If the current directory depth is less than max depth, list contents
        if depth < max_depth:
            for item in directory.iterdir():
                if item.is_dir():
                    summarize_dir_structure(item, depth + 1, max_depth)
                else:
                    # If depth is max depth - 1, show example files instead of all files
                    if depth == max_depth - 1:
                        indent_str = str(item).split('/')[-1].split('_', 1)[1].join(['_', ''])
                        print(f"{indent}    ├── Example files: [station ID]{indent_str}")
                        break  # Only print example files line once per directory
                    else:
                        print(f"{indent}    └── {item.name}")
        else:
            # At max depth, mention that there are more files/directories without listing them
            print(f"{indent}    └── ...")


# Define the path to your dataset directory
CAMELS_ROOT = Path("/content/CAMEL_DATASET/")

# Summarize the directory structure
summarize_dir_structure(CAMELS_ROOT, max_depth=3)

In [ ]:
def load_static_attributes(data_dir: Path, basins: List[str] = []) -> pd.DataFrame:
    """Load static attributes into a pandas DataFrame

    Parameters
    ----------
    data_dir : Path
        Path to the root directory of the dataset. Must contain a folder called 'hydroatlas_attributes' with a file
        called `attributes.csv`. The attributes file is expected to have one column called `basin_id`.
    basins : List[str], optional
        If passed, return only attributes for the basins specified in this list. Otherwise, the attributes of all basins
        are returned.

    Returns
    -------
    pd.DataFrame
        Basin-indexed DataFrame containing the HydroATLAS attributes.
    """
    #LOAD STATIC FEATURES AS DF
    df_veg = pd.read_csv(os.path.join(CAMELS_ROOT,"camels_attributes_v2.0",'camels_vege.txt'),sep=';', dtype = {'gauge_id': object})
    df_topo = pd.read_csv(os.path.join(CAMELS_ROOT,"camels_attributes_v2.0",'camels_topo.txt'),sep=';', dtype = {'gauge_id': object})
    df_soil = pd.read_csv(os.path.join(CAMELS_ROOT,"camels_attributes_v2.0",'camels_soil.txt'),sep=';', dtype = {'gauge_id': object})
    df_hydro = pd.read_csv(os.path.join(CAMELS_ROOT,"camels_attributes_v2.0",'camels_hydro.txt'),sep=';', dtype = {'gauge_id': object})
    df_geol = pd.read_csv(os.path.join(CAMELS_ROOT,"camels_attributes_v2.0",'camels_geol.txt'),sep=';', dtype = {'gauge_id': object})
    df_clim = pd.read_csv(os.path.join(CAMELS_ROOT,"camels_attributes_v2.0",'camels_clim.txt'),sep=';', dtype = {'gauge_id': object})

    data_frames = [df_veg,df_topo,df_soil,df_hydro,df_geol,df_clim]


    from functools import reduce
    df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['gauge_id'],how='outer'), data_frames)
    drop_columns = [
    'area_geospa_fabric', 'geol_1st_class', 'glim_1st_class_frac', 'geol_2nd_class',
        'glim_2nd_class_frac', 'dom_land_cover_frac', 'dom_land_cover', 'high_prec_timing',
        'low_prec_timing', 'q_mean', 'runoff_ratio', 'stream_elas', 'slope_fdc',
        'baseflow_index', 'hfd_mean', 'q5', 'q95', 'high_q_freq', 'high_q_dur', 'low_q_freq',
        'low_q_dur', 'zero_q_freq', 'geol_porostiy', 'root_depth_50', 'root_depth_99', 'organic_frac',
        'water_frac', 'other_frac'
    ]
    df_merged.drop(drop_columns, axis=1, inplace=True)
    basin_attr = df_merged
    if basins:
        basin_attr = basin_attr[basin_attr['gauge_id'] == basins]
    return basin_attr

def load_forcing(basin: str) -> Tuple[pd.DataFrame, int]:
    """Load the meteorological forcing data of a specific basin.

    :param basin: 8-digit code of basin as string.

    :return: pd.DataFrame containing the meteorological forcing data and the
        area of the basin as integer.
    """
    # root directory of meteorological forcings

    forcing_path = CAMELS_ROOT /  'nldas'

    # get path of forcing file
    files = list(glob.glob(f"{str(forcing_path)}/**/{basin}_*.txt"))
    #files = list(os.(f"{str(forcing_path)}/**/{basin}_*.txt"))
    if len(files) == 0:
        raise RuntimeError(f'No forcing file file found for Basin {basin}')
    else:
        file_path = files[0]


    # read-in data and convert date to datetime index
    with open(file_path) as fp:
        df = pd.read_csv(fp, sep='\s+', header=3)
    dates = (df.Year.map(str) + "/" + df.Mnth.map(str) + "/"
             + df.Day.map(str))
    df.index = pd.to_datetime(dates, format="%Y/%m/%d")

    # load area from header
    with open(file_path) as fp:
        content = fp.readlines()
        area = int(content[2])

    return df, area



def load_discharge(basin: str, area: int) ->  pd.Series:
    """Load the discharge time series for a specific basin.

    :param basin: 8-digit code of basin as string.
    :param area: int, area of the catchment in square meters

    :return: A pd.Series containng the catchment normalized discharge.
    """
    # root directory of the streamflow data
    discharge_path = CAMELS_ROOT / 'usgs_streamflow'

    # get path of streamflow file file
    files = list(glob.glob(f"{str(discharge_path)}/**/{basin}_*.txt"))
    if len(files) == 0:
        raise RuntimeError(f'No discharge file found for Basin {basin}')
    else:
        file_path = files[0]

    # read-in data and convert date to datetime index
    col_names = ['basin', 'Year', 'Month', 'Day', 'QObs', 'flag']
    with open(file_path) as fp:
        df = pd.read_csv(fp, sep='\s+', header=None, names=col_names)
    dates = (df.Year.map(str) + "/" + df.Month.map(str) + "/"
             + df.Day.map(str))
    df.index = pd.to_datetime(dates, format="%Y/%m/%d")

    # normalize discharge from cubic feed per second to mm per day
    df.QObs = 28316846.592 * df.QObs * 86400 / (area * 10 ** 6)

    return df.QObs

In [ ]:
def download_basin_list(url: str, save_path: str) -> None:
    """Downloads the basin list from a given URL and saves it to a specified path."""
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raises an HTTPError if the status is 4xx or 5xx
        with open(save_path, "wb") as f:
            f.write(response.content)
    except requests.RequestException as e:
        print(f"Error downloading file: {e}")

def read_basin_data(filepath: str) -> Tuple[List[str], Dict[str, List]]:
    """Reads basin data from a file, loads forcing and discharge data for each basin."""
    basin_name_to_driver = {}
    basin_list = []
    try:
      with open(filepath, 'r') as file:
          for line in file:
              basin_name = line.strip()
              basin_list.append(basin_name)
              forcing, area = load_forcing(basin_name)

              forcing['QObs(mm/d)'] = load_discharge(basin_name, area)
              basin_name_to_driver[basin_name] = [forcing, area]
    except FileNotFoundError:
        print(f"File not found: {filepath}")
        return [], {}
    except Exception as e:
        print(f"An error occurred: {e}")
        return [], {}

    return basin_list, basin_name_to_driver


# Usage
url = "https://raw.githubusercontent.com/neuralhydrology/neuralhydrology/master/examples/06-Finetuning/531_basin_list.txt"
save_path = '/content/CAMEL_DATASET/531_basin_list.txt'
download_basin_list(url, save_path)
basin_list, basin_name_to_driver = read_basin_data(save_path)


In [ ]:
basin_index = 0
forcing, area = basin_name_to_driver[basin_list[basin_index]]
forcing


In [ ]:
year_counts = forcing['Year'].value_counts().sort_index()

# Now plot the data using a bar chart.
year_counts.plot(kind='bar', title='Datapoints per Year')

# Improve the plot appearance.
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.xlabel('Year')  # Adding x-axis label.
plt.ylabel('Count')  # Adding y-axis label.

plt.show()

In [ ]:
def plot_time_series_for_five_years(dataframe, column_name, y_label, column_desc, figsize=(18, 6)):
    """
    Plot time series data for three years from a specified column of a DataFrame.

    Parameters:
    - dataframe: pandas DataFrame containing the data.
    - column_name: String specifying the column to plot.
    - figsize: Tuple specifying the figure size, default is (12, 6).
    """
    plt.figure(figsize=figsize)

    # Plot the data as line plots for each year.
    for year in range(5):  # Assuming the first three years in the dataset.
        start_day = year * 365
        end_day = start_day + 365
        series = dataframe[column_name][start_day:end_day]

        series.plot(kind='line', label=f'Year {year+1}')

    # Improve the plot appearance.
    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['right'].set_visible(False)
    plt.xlabel('Day of the Year')  # More specific label.
    plt.ylabel(f'{y_label}')  # Generic label, adjust as needed.
    plt.title(f'{column_desc}')  # Dynamic title based on column name.
    plt.legend()  # Add a legend to distinguish between years.

    plt.show()

# Plotting Forcings
plot_time_series_for_five_years(forcing, 'SRAD(W/m2)','Solar Radiation (W/m2)','Annual Solar Radiation Comparison')
plot_time_series_for_five_years(forcing, 'PRCP(mm/day)','Precipitation (mm/day)','Annual Precipitation Sum Comparison')
plot_time_series_for_five_years(forcing, 'Dayl(s)','Daylight Duration (s)','Daylight Duration Over Time')
plot_time_series_for_five_years(forcing, 'SWE(mm)','Snow Water Equivalent (mm)','Annual Snow Water Equivalent Comparison')
plot_time_series_for_five_years(forcing, 'Tmax(C)','Maximum Temperature (°C)','Annual Maximum Temperature Comparison')
plot_time_series_for_five_years(forcing, 'Tmin(C)','Minimum Temperature (°C)','Annual  Minimum Temperature  Comparison')
plot_time_series_for_five_years(forcing, 'Vp(Pa)','Vapor Pressure (Pa)','Annual Vapor Pressure Comparison')

plot_time_series_for_five_years(forcing, 'QObs(mm/d)','Observed Discharge (mm/day)','Annual Observed Discharge Comparison')


<body>
    <h2>Hydrological Basin Attributes</h2>
    <table>
        <tr>
            <th>Index</th>
            <th>Column Name</th>
            <th>Description</th>
        </tr>
        <tr>
            <td>1</td>
            <td>gauge_id</td>
            <td>Unique identifier for the gauge station.</td>
        </tr>
        <tr>
            <td>2</td>
            <td>frac_forest</td>
            <td>Fraction of the basin area covered by forest.</td>
        </tr>
        <tr>
            <td>3</td>
            <td>lai_max</td>
            <td>Maximum Leaf Area Index observed in the basin.</td>
        </tr>
        <tr>
            <td>4</td>
            <td>lai_diff</td>
            <td>Variation in Leaf Area Index across different seasons or years.</td>
        </tr>
        <tr>
            <td>5</td>
            <td>gvf_max</td>
            <td>Maximum Green Vegetation Fraction, indicating peak vegetation coverage.</td>
        </tr>
        <tr>
            <td>6</td>
            <td>gvf_diff</td>
            <td>Change in Green Vegetation Fraction between seasons or over years.</td>
        </tr>
        <tr>
            <td>7</td>
            <td>gauge_lat</td>
            <td>Latitude of the gauge station.</td>
        </tr>
        <tr>
            <td>8</td>
            <td>gauge_lon</td>
            <td>Longitude of the gauge station.</td>
        </tr>
        <tr>
            <td>9</td>
            <td>elev_mean</td>
            <td>Average elevation of the basin area.</td>
        </tr>
        <tr>
            <td>10</td>
            <td>slope_mean</td>
            <td>Average slope angle of the basin terrain.</td>
        </tr>
        <tr>
            <td>11</td>
            <td>area_gages2</td>
            <td>Area of the basin covered by the GAGES-II study or similar.</td>
        </tr>
        <tr>
            <td>12</td>
            <td>soil_depth_pelletier</td>
            <td>Soil depth estimates according to Pelletier method or dataset.</td>
        </tr>
        <tr>
            <td>13</td>
            <td>soil_depth_statsgo</td>
            <td>Soil depth estimates according to STATSGO database.</td>
        </tr>
        <tr>
            <td>14</td>
            <td>soil_porosity</td>
            <td>Measure of how porous the soil is within the basin.</td>
        </tr>
        <tr>
            <td>15</td>
            <td>soil_conductivity</td>
            <td>Soil's conductivity, indicating how easily water flows through the soil layers.</td>
        </tr>
        <tr>
            <td>16</td>
            <td>max_water_content</td>
            <td>Maximum water content the soil can hold.</td>
        </tr>
        <tr>
            <td>17</td>
            <td>sand_frac</td>
            <td>Fraction of the soil composed of sand particles.</td>
        </tr>
        <tr>
            <td>18</td>
            <td>silt_frac</td>
            <td>Fraction of the soil made up of silt particles.</td>
        </tr>
        <tr>
            <td>19</td>
            <td>clay_frac</td>
            <td>Fraction of the soil consisting of clay particles.</td>
        </tr>
        <tr>
            <td>20</td>
            <td>carbonate_rocks_frac</td>
            <td>Fraction of the basin's geological substrate made up of carbonate rocks.</td>
        </tr>
        <tr>
            <td>21</td>
            <td>geol_permeability</td>
            <td>Geological permeability, indicating the ability of rock layers to transmit water.</td>
        </tr>
        <tr>
            <td>22</td>
            <td>p_mean</td>
            <td>Average precipitation in the basin.</td>
        </tr>
        <tr>
            <td>23</td>
            <td>pet_mean</td>
            <td>Average potential evapotranspiration in the basin.</td>
        </tr>
        <tr>
            <td>24</td>
            <td>p_seasonality</td>
            <td>Variability in precipitation over different seasons.</td>
        </tr>
        <tr>
            <td>25</td>
            <td>frac_snow</td>
            <td>Fraction of precipitation that falls as snow.</td>
        </tr>
        <tr>
            <td>26</td>
            <td>aridity</td>
            <td>Aridity index, reflecting the dryness of the basin climate.</td>
        </tr>
        <tr>
            <td>27</td>
            <td>high_prec_freq</td>
            <td>Frequency of high precipitation events.</td>
        </tr>
        <tr>
            <td>28</td>
            <td>high_prec_dur</td>
            <td>Duration of high precipitation events.</td>
        </tr>
        <tr>
            <td>29</td>
            <td>low_prec_freq</td>
            <td>Frequency of low precipitation events.</td>
        <tr>
            <td>30</td>
            <td>low_prec_dur</td>
            <td>Duration of low precipitation events.</td>
        </tr>
    </table>
</body>
</html>


In [ ]:
# we visualize the static attributes of the basin.
basin_attr = load_static_attributes(CAMELS_ROOT)
basin_attr

In [ ]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # This line checks if GPU is available

In [ ]:
def reshape_data(x: np.ndarray, y: np.ndarray, seq_length: int) -> Tuple[np.ndarray, np.ndarray]:
    """
    Reshape matrix data into sample shape for LSTM training.

    :param x: Matrix containing input features column wise and time steps row wise
    :param y: Matrix containing the output feature.
    :param seq_length: Length of look back days for one day of prediction

    :return: Two np.ndarrays, the first of shape (samples, length of sequence,
        number of features), containing the input data for the LSTM. The second
        of shape (samples, 1) containing the expected output for each input
        sample.
    """
    num_samples, num_features = x.shape

    x_new = np.zeros((num_samples - seq_length + 1, seq_length, num_features))
    y_new = np.zeros((num_samples - seq_length + 1, 1))

    for i in range(0, x_new.shape[0]):
        x_new[i, :, :num_features] = x[i:i + seq_length, :]
        y_new[i, :] = y[i + seq_length - 1, 0]

    return x_new, y_new

class CamelsTXT(Dataset):
    """Torch Dataset for basic use of data from the CAMELS data set.

    This data set provides meteorological observations and discharge of a given
    basin from the CAMELS data set.
    """

    def __init__(self, basins: List=[], seq_length: int=365,period: str=None,
                 dates: List=None, means: pd.Series=None, stds: pd.Series=None, z_means: pd.Series=None, z_stds: pd.Series=None, normalize_input= True):
        """Initialize Dataset containing the data of a single basin.

        :param basin: 8-digit code of list of basins as string.
        :param seq_length: (optional) Length of the time window of
            meteorological input provided for one time step of prediction.
        :param period: (optional) One of ['train', 'eval']. None loads the
            entire time series.
        :param dates: (optional) List of pd.DateTimes of the start and end date
            of the discharge period that is used.
        :param means: (optional) Means of input and output features derived from
            the training period. Has to be provided for 'eval' period. Can be
            retrieved if calling .get_means() on the data set.
        :param stds: (optional) Stds of input and output features derived from
            the training period. Has to be provided for 'eval' period. Can be
            retrieved if calling .get_stds() on the data set.
        """
        self.basins = basins
        self.seq_length = seq_length
        self.period = period
        self.dates = dates
        self.means = means
        self.stds = stds
        self.normalize_input = normalize_input

        self.z_means = z_means
        self.z_stds = z_stds

        self.x_all =[]
        self.y_all = []
        self.z_all = []

        # # load data into memory
        # self.basin = self.basins[0]

        #calculate mean and std
        self.calculate_mean_std()

        for basin in self.basins:
          self.basin = basin
          self.x, self.y , self.z = self._load_data()
          self.x_all.append(self.x)
          self.y_all.append(self.y)
          self.z_all.append(self.z)


        self.x_all = torch.cat(self.x_all,dim=0)
        self.y_all = torch.cat(self.y_all,dim=0)
        self.z_all = torch.cat(self.z_all,dim=0)


        # print("x,y shape after load", self.x.shape, self.y.shape)
        # store number of samples as class attribute
        self.num_samples = self.x_all.shape[0]

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx: int):
        return self.x_all[idx], self.y_all[idx], self.z_all[idx]

    def calculate_mean_std(self):
      # # if training period store means and stds
      if self.period=='train':
        df_all = []
        basin_attr_all =[]
        for basin in self.basins:
          self.basin = basin
          df, area = load_forcing(self.basin)
          df['QObs(mm/d)'] = load_discharge(self.basin, area)
          basin_attr = load_static_attributes(self.basin)
          basin_attr = basin_attr.drop(['gauge_id'], axis=1)
          df_all.append(df)
          basin_attr_all.append(basin_attr)

        df = pd.concat(df_all)
        basin_attr = pd.concat(basin_attr_all)

        # print("basin_attr all", basin_attr)
        self.means = df.mean()
        self.stds = df.std()
        self.z_means = basin_attr.mean()
        self.z_stds = basin_attr.std()


    def _load_data(self):
        """Load input and output data from text files."""
        df, area = load_forcing(self.basin)
        df['QObs(mm/d)'] = load_discharge(self.basin, area)

        basin_attr = load_static_attributes(self.basin)
        basin_attr = basin_attr.drop(['gauge_id'], axis=1)

        if self.dates is not None:
            # If meteorological observations exist before start date
            # use these as well. Similiar to hydrological warmup period.
            if self.dates[0] - pd.DateOffset(days=self.seq_length) > df.index[0]:
                start_date = self.dates[0] - pd.DateOffset(days=self.seq_length)
            else:
                start_date = self.dates[0]
            df = df[start_date:self.dates[1]]

        # # if training period store means and stds
        # if self.period == 'train':
        #     self.means = df.mean()
        #     self.stds = df.std()
        #     self.z_means = basin_attr.mean()
        #     self.z_stds = basin_attr.std()


        # extract input and output features from DataFrame
        x = np.array([df['PRCP(mm/day)'].values,
                      df['SRAD(W/m2)'].values,
                      df['Tmax(C)'].values,
                      df['Tmin(C)'].values,
                      df['Vp(Pa)'].values]).T
        z = basin_attr.values


        y = np.array([df['QObs(mm/d)'].values]).T

        # normalize data, reshape for LSTM training and remove invalid samples

        if self.normalize_input:
          x = self._local_normalization(x, variable='inputs')
          z = self._local_normalization(z, variable='z')
        else:
          x = self._local_normalization(x, variable='non_mass_input')
          z = self._local_normalization(z, variable='z')


        x, y = reshape_data(x, y, self.seq_length)

        if self.period == "train":
            # Delete all samples, where discharge is NaN
            if np.sum(np.isnan(y)) > 0:
                print(f"Deleted some records because of NaNs {self.basin}")
                x = np.delete(x, np.argwhere(np.isnan(y)), axis=0)
                y = np.delete(y, np.argwhere(np.isnan(y)), axis=0)

            # Deletes all records, where no discharge was measured (-999)
            x = np.delete(x, np.argwhere(y < 0)[:, 0], axis=0)
            y = np.delete(y, np.argwhere(y < 0)[:, 0], axis=0)

            # normalize discharge
            if self.normalize_input:
              y = self._local_normalization(y, variable='output')

        # convert arrays to torch tensors
        x = torch.from_numpy(x.astype(np.float32))
        y = torch.from_numpy(y.astype(np.float32))


        z = torch.from_numpy(z.astype(np.float32))
        z= z.repeat(x.shape[0],1)

        return x, y, z

    def _local_normalization(self, feature: np.ndarray, variable: str) -> \
            np.ndarray:
        """Normalize input/output features with local mean/std.

        :param feature: Numpy array containing the feature(s) as matrix.
        :param variable: Either 'inputs' or 'output' showing which feature will
            be normalized
        :return: array containing the normalized feature
        """
        if variable == 'inputs':
            means = np.array([self.means['PRCP(mm/day)'],
                              self.means['SRAD(W/m2)'],
                              self.means['Tmax(C)'],
                              self.means['Tmin(C)'],
                              self.means['Vp(Pa)']])
            stds = np.array([self.stds['PRCP(mm/day)'],
                             self.stds['SRAD(W/m2)'],
                             self.stds['Tmax(C)'],
                             self.stds['Tmin(C)'],
                             self.stds['Vp(Pa)']])
            feature = (feature - means) / stds
        elif variable == 'non_mass_input':
            means = np.array([0,   # no normalization for mass input
                              self.means['SRAD(W/m2)'],
                              self.means['Tmax(C)'],
                              self.means['Tmin(C)'],
                              self.means['Vp(Pa)']])
            stds = np.array([1,   # no normalization for mass input
                            self.stds['SRAD(W/m2)'],
                             self.stds['Tmax(C)'],
                             self.stds['Tmin(C)'],
                             self.stds['Vp(Pa)']])
            feature = (feature - means) / stds
        elif variable == 'output':
            feature = ((feature - self.means["QObs(mm/d)"]) /
                       self.stds["QObs(mm/d)"])
        elif variable == 'z':
            feature = (feature - self.z_means.values) / self.z_stds.values
        else:
            raise RuntimeError(f"Unknown variable type {variable}")

        return feature

    def local_rescale(self, feature: np.ndarray, variable: str) -> \
            np.ndarray:
        """Rescale input/output features with local mean/std.

        :param feature: Numpy array containing the feature(s) as matrix.
        :param variable: Either 'inputs' or 'output' showing which feature will
            be normalized
        :return: array containing the normalized feature
        """
        if variable == 'inputs':
            means = np.array([self.means['PRCP(mm/day)'],
                              self.means['SRAD(W/m2)'],
                              self.means['Tmax(C)'],
                              self.means['Tmin(C)'],
                              self.means['Vp(Pa)']])
            stds = np.array([self.stds['PRCP(mm/day)'],
                             self.stds['SRAD(W/m2)'],
                             self.stds['Tmax(C)'],
                             self.stds['Tmin(C)'],
                             self.stds['Vp(Pa)']])
            feature = feature * stds + means
        elif variable == 'output':
            feature = (feature * self.stds["QObs(mm/d)"] +
                       self.means["QObs(mm/d)"])
        elif variable == 'z':
            feature = ((feature * self.z_stds.values + self.z_means.values) )
        else:
            raise RuntimeError(f"Unknown variable type {variable}")

        return feature

    def get_means(self):
        return self.means

    def get_stds(self):
        return self.stds

    def get_z_means(self):
        return self.z_means

    def get_z_stds(self):
        return self.z_stds

class Model(nn.Module):
    """Implementation of a single layer LSTM network"""

    def __init__(self, hidden_size: int, input_size:int =5, dropout_rate: float=0.0):
        """Initialize model

        :param hidden_size: Number of hidden units/LSTM cells
        :param dropout_rate: Dropout rate of the last fully connected
            layer. Default 0.0
        """
        super(Model, self).__init__()
        self.hidden_size = hidden_size
        self.dropout_rate = dropout_rate
        self.input_size = input_size

        # create required layer
        self.lstm = nn.LSTM(input_size=self.input_size, hidden_size=self.hidden_size,
                            num_layers=1, bias=True, batch_first=True)
        self.dropout = nn.Dropout(p=self.dropout_rate)
        self.fc = nn.Linear(in_features=self.hidden_size, out_features=1)

    def forward(self, x: torch.Tensor,  z: torch.Tensor = None,with_static: bool = False, static_concatenate: bool = False) -> torch.Tensor:
        """Forward pass through the Network.

        :param x: Tensor of shape [batch size, seq length, num features]
            containing the input data for the LSTM network.

        :return: Tensor containing the network predictions
        """

        input = x
        if with_static:
          if static_concatenate:
            z_repeat= z.unsqueeze(1).repeat(1, x.shape[1], 1)
            input = torch.cat((x, z_repeat), -1)


        output, (h_n, c_n) = self.lstm(input)

        # perform prediction only at the end of the input sequence
        pred = self.fc(self.dropout(h_n[-1,:,:]))
        return pred

def train_epoch(model, optimizer, loader, loss_func, epoch, with_static=False, static_concatenate=True):
    """Train model for a single epoch.

    :param model: A torch.nn.Module implementing the LSTM model
    :param optimizer: One of PyTorchs optimizer classes.
    :param loader: A PyTorch DataLoader, providing the trainings
        data in mini batches.
    :param loss_func: The loss function to minimize.
    :param epoch: The current epoch (int) used for the progress bar
    """
    # set model to train mode (important for dropout)
    model.train()
    pbar = tqdm.tqdm_notebook(loader)
    pbar.set_description(f"Epoch {epoch}")
    # request mini-batch of data from the loader
    for xs, ys, zs in pbar:
        # delete previously stored gradients from the model
        optimizer.zero_grad()

        if not with_static:
          # push data to GPU (if available)
          xs, ys = xs.to(DEVICE), ys.to(DEVICE)
          # get model predictions
          y_hat = model(xs)
        else:
          # push data to GPU (if available)
          xs, ys, zs = xs.to(DEVICE), ys.to(DEVICE), zs.to(DEVICE)
          # get model predictions
          y_hat = model(xs, zs, with_static, static_concatenate)

        # calculate loss
        loss = loss_func(y_hat, ys)
        # calculate gradients
        loss.backward()
        # update the weights
        optimizer.step()
        # write current loss in the progress bar
        pbar.set_postfix_str(f"Loss: {loss.item():.4f}")


def eval_model(model, loader, with_static=False, static_concatenate=True) -> Tuple[torch.Tensor, torch.Tensor]:
    """Evaluate the model.

    :param model: A torch.nn.Module implementing the LSTM model
    :param loader: A PyTorch DataLoader, providing the data.

    :return: Two torch Tensors, containing the observations and
        model predictions
    """
    # set model to eval mode (important for dropout)
    model.eval()
    obs = []
    preds = []
    # in inference mode, we don't need to store intermediate steps for
    # backprob
    with torch.no_grad():
        # request mini-batch of data from the loader
        for xs, ys, zs in loader:
            # push data to GPU (if available)
            if not with_static:
              xs = xs.to(DEVICE)
              # get model predictions
              y_hat = model(xs)
              obs.append(ys)
              preds.append(y_hat)
            else:
              xs, zs = xs.to(DEVICE), zs.to(DEVICE)
              # get model predictions
              y_hat = model(xs, zs, with_static, static_concatenate)
              obs.append(ys)
              preds.append(y_hat)
    return torch.cat(obs), torch.cat(preds)

def calc_nse(obs: np.array, sim: np.array) -> float:
    """Calculate Nash-Sutcliff-Efficiency.

    :param obs: Array containing the observations
    :param sim: Array containing the simulations
    :return: NSE value.
    """
    # only consider time steps, where observations are available
    sim = np.delete(sim, np.argwhere(obs < 0), axis=0)
    obs = np.delete(obs, np.argwhere(obs < 0), axis=0)

    # check for NaNs in observations
    sim = np.delete(sim, np.argwhere(np.isnan(obs)), axis=0)
    obs = np.delete(obs, np.argwhere(np.isnan(obs)), axis=0)

    denominator = np.sum((obs - np.mean(obs)) ** 2)
    numerator = np.sum((sim - obs) ** 2)
    nse_val = 1 - numerator / denominator

    return nse_val

In [ ]:
basins = ['01022500']# can be changed to a list any 8-digit basin id contained in the CAMELS data set
hidden_size = 32 # Number of LSTM cells
dropout_rate = 0.2 # Dropout rate of the final fully connected Layer [0.0, 1.0]
learning_rate = 1e-3 # Learning rate used to update the weights
sequence_length = 365 # Length of the meteorological record provided to the network

##############
# Data set up#
##############

# Training data
start_date = pd.to_datetime("1980-10-01", format="%Y-%m-%d")
end_date = pd.to_datetime("1995-09-30", format="%Y-%m-%d")
ds_train = CamelsTXT(basins, seq_length=sequence_length, period="train", dates=[start_date, end_date])
tr_loader = DataLoader(ds_train, batch_size=256, shuffle=True)

# Validation data. We use the feature means/stds of the training period for normalization
means = ds_train.get_means()
stds = ds_train.get_stds()
z_means = ds_train.get_z_means()
z_stds = ds_train.get_z_stds()
start_date = pd.to_datetime("1995-10-01", format="%Y-%m-%d")
end_date = pd.to_datetime("2000-09-30", format="%Y-%m-%d")
ds_val = CamelsTXT(basins, seq_length=sequence_length, period="eval", dates=[start_date, end_date],
                     means=means, stds=stds, z_means=z_means, z_stds=z_stds)
val_loader = DataLoader(ds_val, batch_size=2048, shuffle=False)

# Test data. We use the feature means/stds of the training period for normalization
start_date = pd.to_datetime("2000-10-01", format="%Y-%m-%d")
end_date = pd.to_datetime("2010-09-30", format="%Y-%m-%d")
ds_test = CamelsTXT(basins, seq_length=sequence_length, period="eval", dates=[start_date, end_date],
                     means=means, stds=stds, z_means=z_means, z_stds=z_stds)
test_loader = DataLoader(ds_test, batch_size=2048, shuffle=False)


#########################
# Model, Optimizer, Loss#
#########################

# Here we create our model, feel free
model = Model(hidden_size=hidden_size, dropout_rate=dropout_rate).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_func = nn.MSELoss()

In [ ]:
n_epochs = 10 # Number of training epochs

for i in range(n_epochs):
    train_epoch(model, optimizer, tr_loader, loss_func, i+1)
    obs, preds = eval_model(model, val_loader)
    preds = ds_val.local_rescale(preds.numpy(), variable='output')
    nse = calc_nse(obs.numpy(), preds)
    tqdm.tqdm.write(f"Validation NSE: {nse:.2f}")

In [ ]:
def plot_results(date_range, observations, predictions, basin, nse):
    """Plot the observed and predicted discharges over the date range"""
    # Set the plot style
    sns.set_style("whitegrid")

    # Create a figure and a single subplot
    fig, ax = plt.subplots(figsize=(16, 6))

    # Plot observations and predictions
    ax.plot(date_range, observations, label="Observation", color='blue', linewidth=1, linestyle='-', marker='o', markersize=2)
    ax.plot(date_range, predictions, label="Prediction", color='darkorange', linewidth=1, linestyle='--', marker='x', markersize=2)

    # Add legend with custom fontsize
    ax.legend(fontsize='large')

    # Set title with dynamic basin and NSE information
    ax.set_title(f"LSTM (Single Basin) in Basin {basin} - Test set NSE: {nse:.3f}", fontsize=16)

    # Set axis labels with custom fontsize
    ax.set_xlabel("Date", fontsize=14)
    ax.set_ylabel("Discharge (mm/d)", fontsize=14)

    # Customize tick parameters
    ax.tick_params(axis='x', rotation=45)
    ax.tick_params(axis='both', which='major', labelsize=12)

    # Enhance grid visibility
    ax.grid(True, which='both', linestyle='--', linewidth=0.5)

    # Tight layout for better spacing
    plt.tight_layout()

    # Display the plot
    plt.show()

# Evaluate on test set
basin = basins[0]

#Evaluate the model and return observations and predictions
obs, preds = eval_model(model, test_loader)

#Rescale predictions to their original scale.
preds_rescaled = ds_val.local_rescale(preds.numpy(), variable='output')

#Calculate the Nash-Sutcliffe Efficiency.
nse = calc_nse(obs.numpy(), preds_rescaled)

# Define date range for plotting
start_date = ds_test.dates[0]
end_date = ds_test.dates[-1]  # Assuming 'dates' is a list of all dates in the test set
date_range = pd.date_range(start_date, end_date + pd.DateOffset(days=1))

# Plot results
plot_results(date_range, obs, preds_rescaled, basin, nse)

In [ ]:
class TransformerModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_heads, dropout_rate):
        super(TransformerModel, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.num_heads = num_heads
        self.dropout_rate = dropout_rate

        # Projection layer to match input size to hidden size
        self.projection = nn.Linear(self.input_size, self.hidden_size)

        # Define the Transformer layers
        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model=self.hidden_size,
            nhead=self.num_heads,
            dropout=self.dropout_rate
        )

        self.transformer_encoder = nn.TransformerEncoder(
            self.encoder_layer,
            num_layers=self.num_layers
        )

        # Final fully connected layer
        self.fc = nn.Linear(self.hidden_size, 1)  # Output for regression (1 value)

    def forward(self, x):
        # x: (batch_size, seq_length, input_size)

        # Project the input to match the hidden_size
        x = self.projection(x)  # (batch_size, seq_length, hidden_size)

        # Permute for transformer input shape (seq_length, batch_size, hidden_size)
        x = x.permute(1, 0, 2)

        # Pass through transformer encoder
        transformer_out = self.transformer_encoder(x)

        # Get the output from the last time step (or aggregate across all time steps)
        out = transformer_out[-1, :, :]  # We can use the last time step for prediction

        # Pass through fully connected layer
        output = self.fc(out)
        return output


In [ ]:
# Define model parameters
input_size = 5  # You can add any additional channels if needed
hidden_size = 16  # Number of hidden units or addd projection layer
num_layers = 5  # Number of Transformer layers
num_heads = 4  # Number of attention heads
dropout_rate = 0.1  # Dropout rate for Transformer layers

basins = ['01022500']# can be changed to a list any 8-digit basin id contained in the CAMELS data set
dropout_rate = 0.1 # Dropout rate of the final fully connected Layer [0.0, 1.0]
learning_rate = 1e-3 # Learning rate used to update the weights
sequence_length = 365 # Length of the meteorological record provided to the network

##############
# Data set up#
##############

# Training data
start_date = pd.to_datetime("1980-10-01", format="%Y-%m-%d")
end_date = pd.to_datetime("1995-09-30", format="%Y-%m-%d")
ds_train = CamelsTXT(basins, seq_length=sequence_length, period="train", dates=[start_date, end_date])
tr_loader = DataLoader(ds_train, batch_size=256, shuffle=True)

# Validation data. We use the feature means/stds of the training period for normalization
means = ds_train.get_means()
stds = ds_train.get_stds()
z_means = ds_train.get_z_means()
z_stds = ds_train.get_z_stds()
start_date = pd.to_datetime("1995-10-01", format="%Y-%m-%d")
end_date = pd.to_datetime("2000-09-30", format="%Y-%m-%d")
ds_val = CamelsTXT(basins, seq_length=sequence_length, period="eval", dates=[start_date, end_date],
                     means=means, stds=stds, z_means=z_means, z_stds=z_stds)
val_loader = DataLoader(ds_val, batch_size=2048, shuffle=False)

# Test data. We use the feature means/stds of the training period for normalization
start_date = pd.to_datetime("2000-10-01", format="%Y-%m-%d")
end_date = pd.to_datetime("2010-09-30", format="%Y-%m-%d")
ds_test = CamelsTXT(basins, seq_length=sequence_length, period="eval", dates=[start_date, end_date],
                     means=means, stds=stds, z_means=z_means, z_stds=z_stds)
test_loader = DataLoader(ds_test, batch_size=2048, shuffle=False)


#########################
# Model, Optimizer, Loss#
#########################

# Here we create our model, feel free
# model = Model(hidden_size=hidden_size, dropout_rate=dropout_rate).to(DEVICE)
# Initialize the Transformer model
model = TransformerModel(input_size=input_size,
                         hidden_size=hidden_size,
                         num_layers=num_layers,
                         num_heads=num_heads,
                         dropout_rate=dropout_rate).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_func = nn.MSELoss()

In [ ]:
n_epochs = 7 # Number of training epochs

for i in range(n_epochs):
    train_epoch(model, optimizer, tr_loader, loss_func, i+1)
    obs, preds = eval_model(model, val_loader)
    preds = ds_val.local_rescale(preds.numpy(), variable='output')
    nse = calc_nse(obs.numpy(), preds)
    tqdm.tqdm.write(f"Validation NSE: {nse:.2f}")


In [ ]:
basins = ['01022500','01013500', '06289000','08271000','11451100']# can be changed to a list any 8-digit basin id contained in the CAMELS data set
hidden_size = 16 # Number of LSTM cells
dropout_rate = 0.0 # Dropout rate of the final fully connected Layer [0.0, 1.0]
learning_rate = 1e-3 # Learning rate used to update the weights
sequence_length = 365 # Length of the meteorological record provided to the network

##############
# Data set up#
##############

# Training data
start_date = pd.to_datetime("1980-10-01", format="%Y-%m-%d")
end_date = pd.to_datetime("1995-09-30", format="%Y-%m-%d")
ds_train = CamelsTXT(basins, seq_length=sequence_length, period="train", dates=[start_date, end_date])
tr_loader = DataLoader(ds_train, batch_size=256, shuffle=True)

# Validation data. We use the feature means/stds of the training period for normalization
means = ds_train.get_means()
stds = ds_train.get_stds()
z_means = ds_train.get_z_means()
z_stds = ds_train.get_z_stds()
start_date = pd.to_datetime("1995-10-01", format="%Y-%m-%d")
end_date = pd.to_datetime("2000-09-30", format="%Y-%m-%d")
ds_val = CamelsTXT(basins, seq_length=sequence_length, period="eval", dates=[start_date, end_date],
                     means=means, stds=stds, z_means=z_means, z_stds=z_stds)
val_loader = DataLoader(ds_val, batch_size=2048, shuffle=False)

# Test data. We use the feature means/stds of the training period for normalization
start_date = pd.to_datetime("2000-10-01", format="%Y-%m-%d")
end_date = pd.to_datetime("2010-09-30", format="%Y-%m-%d")
ds_test = CamelsTXT(basins, seq_length=sequence_length, period="eval", dates=[start_date, end_date],
                     means=means, stds=stds, z_means=z_means, z_stds=z_stds)
test_loader = DataLoader(ds_test, batch_size=2048, shuffle=False)



#########################
# Model, Optimizer, Loss#
#########################

# Here we create our model, feel free
model = Model(hidden_size=hidden_size, dropout_rate=dropout_rate).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_func = nn.MSELoss()

In [ ]:
n_epochs = 7 # Number of training epochs

for i in range(n_epochs):
    train_epoch(model, optimizer, tr_loader, loss_func, i+1)
    obs, preds = eval_model(model, val_loader)
    preds = ds_val.local_rescale(preds.numpy(), variable='output')
    nse = calc_nse(obs.numpy(), preds)
    tqdm.tqdm.write(f"Validation NSE: {nse:.2f}")

In [ ]:
n_epochs = 7 # Number of training epochs

for i in range(n_epochs):
    train_epoch(model, optimizer, tr_loader, loss_func, i+1)
    obs, preds = eval_model(model, val_loader)
    preds = ds_val.local_rescale(preds.numpy(), variable='output')
    nse = calc_nse(obs.numpy(), preds)
    tqdm.tqdm.write(f"Validation NSE: {nse:.2f}")

In [ ]:
# test_basins = ['01022500','01013500', '06289000','08271000','11451100']
test_basins = ['01022500','01047000']
for basin in test_basins:
    basins = [basin]

    ds_test = CamelsTXT(basins, seq_length=sequence_length, period="eval", dates=[start_date, end_date],
                        means=means, stds=stds, z_means=z_means, z_stds=z_stds)
    test_loader = DataLoader(ds_test, batch_size=2048, shuffle=False)


    #Evaluate the model and return observations and predictions
    obs, preds = eval_model(model, test_loader)

    #Rescale predictions to their original scale.
    preds_rescaled = ds_val.local_rescale(preds.numpy(), variable='output')

    #Calculate the Nash-Sutcliffe Efficiency.
    nse = calc_nse(obs.numpy(), preds_rescaled)

    # Define date range for plotting
    start_date = ds_test.dates[0]
    end_date = ds_test.dates[-1]  # Assuming 'dates' is a list of all dates in the test set
    date_range = pd.date_range(start_date, end_date + pd.DateOffset(days=1))

    # Plot results
    plot_results(date_range, obs, preds_rescaled, basins, nse)

In [ ]:
import math

class PositionalEncoding(nn.Module): # with sinusoidal stuff
    def __init__(self, hidden_size, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, hidden_size)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, hidden_size, 2).float() * (-math.log(10000.0) / hidden_size))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.pe = pe.unsqueeze(0)

    def forward(self, x):
        return x + self.pe[:, :x.size(1)].to(x.device)

In [ ]:
class TransformerModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_heads, dropout):
        super(TransformerModel, self).__init__()
        self.input_fc = nn.Linear(input_size, hidden_size)  # Project input to hidden size
        self.positional_encoding = PositionalEncoding(hidden_size) # Learnable Positional Encoding

        encoder_layers = nn.TransformerEncoderLayer(
            d_model=hidden_size, nhead=num_heads, dim_feedforward=4 * hidden_size, dropout=dropout, batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers=num_layers)
        self.output_fc = nn.Linear(hidden_size, 1)  # Predict discharge (or another target)

    def forward(self, x):
        x = self.input_fc(x)  # Map input to hidden size
        x += self.positional_encoding[:x.shape[1], :].unsqueeze(0)  # Add positional encoding
        x = self.transformer_encoder(x)  # Transformer Encoder
        x = self.output_fc(x[:, -1, :])  # Use last time step output for prediction
        return x

In [ ]:
!git clone -b PI3NN_LSTM_dev https://github.com/liusiyan/UQnet.git

In [ ]:
%cd UQnet

In [ ]:
!python main_PI3NN.py --data ph --mode lstm_encoder --project ./examples/streamflow_proj/ --exp ph --configs ./examples/streamflow_proj/ph/configs_encoder.json

In [ ]:
!python main_PI3NN.py --data ph --mode PI3NN_MLP --project ./examples/streamflow_proj/ --exp ph --configs ./examples/streamflow_proj/ph/configs_PI3NN.json